In [1]:
import pandas as pd

# 데이터 불러오기, \s+ 는 공백이 하나 이상이라는 뜻
feature_name_df = pd.read_csv("../../data files/human_activity/features.txt",sep = '\s+', header = None, names = ['column_index','column_name'])

# 중복된 feature에 인덱스를 붙여 중복을 피하게 하는 함수 생성
def get_new_feature_name(old_feature_name_df):
    # cumcount 함수를 사용하여 몇번째 중복인지 확인. cumcount는 중복이 발생할경우 1씩 올라간다.
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),columns=['dup_cnt'])

    # 인덱스를 새로 추가
    feature_dup_df = feature_dup_df.reset_index()

    # feature name이 들어있는 data frame과 중복 feature 정보가 들어있는 data frame을 index 기준으로 모든 행을 병합
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')

    # 중복되는 feature name에 숫자 부여. 숫자는 dup_cnt에 의해 중복된 순서만큼 부여됨. 예를들어 2번째 중복이면 '_2'가 부여됨
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) if x[1] > 0 else x[0], axis =1)

    # index는 필요없으니 삭제함.
    new_feature_name_df = new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df

# 데이터 로딩
def get_human_dataset():
    # 각 데이터파일들은 공백으로 분리되어 있기 때문에 공백문자를 sep로 할당.
    feature_name_df = pd.read_csv('../../data files/human_activity/features.txt',sep='\s+',header=None,names=['column_index','column_name'])
    
    # 중복된 feature명을 수정
    new_feature_name_df = get_new_feature_name(feature_name_df)

    # data frame에 피처명을 컬럼으로 부여하기위해 리스트 객체로 다시 변환 모든 feature name은 1번째 열에 있음.
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()

    # 학습 feature 데이터 셋과 테스트 feature 데이터를 dataframe으로 로딩. 컬럼은  feature_name 적용
    x_train = pd.read_csv('../../data files/human_activity/train/X_train.txt',sep='\s+',names=feature_name)
    x_test = pd.read_csv('../../data files/human_activity/test/X_test.txt',sep='\s+',names=feature_name)

    # 학습 label과 테스트 label 데이터를 data frame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('../../data files/human_activity/train/y_train.txt',sep='\s+',names=['action'])
    y_test = pd.read_csv('../../data files/human_activity/test/y_test.txt',sep='\s+',names=['action'])

    # 로드된 학습/테스트용 데이터프레임을 모두 반환
    return x_train,x_test,y_train,y_test

# 데이터 준비
x_train, x_test, y_train, y_test = get_human_dataset()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

# 모델이 동작하는 시간 측정
start_time = time.time()

# 모델 학습 및 예측, 평가
gb_clf = GradientBoostingClassifier(random_state=56)
gb_clf.fir(x_train,y_train)
pred = gb_clf.predict(x_test)
gb_accuracy = accuracy_score(y_test,pred)
print('GBM Accuracy : {:.4f}'.format(gb_accuracy))
print('GBM Runtime {:.1f}'.format(time.time() - start_time))